# Libraries

This initial setup is for first use only. It will create an initial database in SQL and do an initial scraping of both tradable stocks in Revolut and stocks on the S&P500. It will then do a first download of financial data and store the metrics to the database

In [1]:
# data science
import pandas as pd
import numpy as np

# finacials
import yfinance as yf
from yahoofinancials import YahooFinancials

# web scraping
import requests
import lxml.html as lh
import re
import unicodedata
import webbrowser

# sql
import mysql.connector

# other
from datetime import datetime
from tqdm.notebook import tqdm, trange

# SQL Connection

In [2]:
db = mysql.connector.connect(host        = "localhost",
                             user        = "root",
                             passwd      = "xxxxxxxx",
                             auth_plugin = "mysql_native_password",
                             database    = "investing"
                            )

my_cursor = db.cursor(buffered = True)

today    = datetime.today()

# Data Collection

## Revolut

In [3]:
# requesting the data
url = "https://globefunder.com/revolut-stocks-list/"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_element = doc.xpath("//tr")

# extracting the columns
col=[]
for tr in tr_element[0]:
    name = tr.text_content()
    name = re.sub('\n', '', name)
    col.append((name,[]))

# extracting the values in the columns
for j in range(1,len(tr_element)):
    T = tr_element[j]
    for i, tr in enumerate(T.iterchildren()):
        data = tr.text_content()
        data = re.sub("\n", "", data)
        data = re.sub("\.", "-", data)
        col[i][1].append(data)

        
# transforming to dataframe
Dict = {title:column for (title,column) in col}
rev   = pd.DataFrame(Dict)
rev.columns = ["#", "company", "ticker", 'Stock price (USD)',"sector_1", "industry", "market"]

# extracting ticker and company name
rev = rev.loc[:, ["ticker", "company", "sector_1"]]

# dictionary of different sectors
sector_dict = {"Industrials"           : "Industrials", 
               "Healthcare"            : "Health Care",
               "Technology"            : "Information Technology",
               "Communication Services": "Communication Services",
               "Consumer Cyclical"     : "Consumer Discretionary",
               "Utilities"             : "Utilities",
               "Financial Services"    : "Financials",
               "Real Estate"           : "Real Estate",
               "Basic Materials"       : "Materials",
               "Consumer Defensive"    : "Consumer Staples",
               "Energy"                : "Energy"
              }

# getting the RIGTH sectors
startTime = datetime.now()
sectors = []

# looping over all the tickers
#for index, value in rev.iterrows():
for index in trange(len(rev)):
    
    # sector one in lower cap
    rev.loc[index, "sector_1"] = str.lower(rev.loc[index, "sector_1"])
    
    # extracting sectors from yf
    sctr = yf.Ticker(rev.loc[index, "ticker"])
    try:
        sctr = sctr.info["sector"]
        
        # transforming sectors
        for k in sector_dict.keys():
            if k == sctr:
                sctr = sector_dict[k]
        sectors.append(sctr)
        
    # if no sectors, unknown
    except:
        sectors.append("UNKNOWN")

        
# creating a second sector column
rev["sector_2"] = sectors

# accounting for unknown sectors from previous task
last_sect = []

for index, value in rev.iterrows():
    
    # comparing unknowns original sector with all companies within same sector
    if rev.loc[index,"sector_2"] == "UNKNOWN":
        org_sect = [rev.loc[index,"sector_1"]][0]
        new_sect = rev[rev.loc[:,"sector_1"] == org_sect]["sector_2"].value_counts().idxmax()
        last_sect.append(new_sect)
        
    else:
        last_sect.append(rev.loc[index,"sector_2"])
rev["sector_3"] = last_sect

# tossing previous sector columns
rev = rev.loc[:,["ticker", "company", "sector_3"]]
rev.columns = ["ticker", "company", "sector"]

## S&P500

In [4]:
# requesting the data
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_element = doc.xpath("//tr")

# extracting the columns
col=[]
for tr in tr_element[0]:
    name = tr.text_content()
    name = re.sub('\n', '', name)
    col.append((name,[]))
    
# extracting the values for CURRENT constituents
for j in range(1,506):
    T = tr_element[j]
    for i, tr in enumerate(T.iterchildren()):
        data = tr.text_content()
        data = re.sub("\n", "", data)
        data = re.sub("\.", "-", data)
        col[i][1].append(data)

        
# transforming to dataframe
Dict = {title:column for (title,column) in col[:4]}
sp   = pd.DataFrame(Dict)
sp.columns = ["ticker", "company", "sec_filing", "sector"]

sp = sp.loc[:, ["ticker", "company", "sector"]]

## Merging the two Datasets

In [5]:
# concatenating the two stock lists
full_stk = pd.concat([rev, sp])

# dropping duplicates, keeping revolut
full_stk = full_stk.drop_duplicates(subset = "ticker", keep = "last")

# resetting index
full_stk = full_stk.reset_index(drop = True)

# declaring affiliation
sp["SP"] = 1
rev["Revolut"] = 1
full_stk = full_stk.merge(sp.loc[:,["ticker", "SP"]], on = "ticker", how = "outer")
full_stk = full_stk.merge(rev.loc[:,["ticker", "Revolut"]], on = "ticker", how = "outer")

# replacing nan with 0
full_stk = full_stk.fillna(value = 0)

## Financial Information

In [6]:
startTime = datetime.now()

# list of stocks to loop over
stocks = list(full_stk["ticker"])


# empty list for failed extractions
fails  = []

# empty item for df
all_stats = None

# looping over stocks
for i in trange(len(stocks)):
    
    try:
        # downloading statistical data
        stock_stats = pd.json_normalize(YahooFinancials(stocks[i]).get_key_statistics_data()[stocks[i]])

        # add additional metrics

        stock_stats["pricetoSales"] = YahooFinancials(stocks[i]).get_price_to_sales()
        
        stock_stats["ticker"]       = stocks[i]


        # either create or add to df
        if all_stats is None:
            all_stats = stock_stats
        else:
            all_stats = pd.concat([all_stats, stock_stats], axis = 0)

    
    except:
        
        # listing failed attempts
        fails.append(stocks[i])




Starting Financial Download



In [7]:
# making copy of data
stats = all_stats.copy()

# converting to numeric values
for i in stats:
    try:
        stats[i] = pd.to_numeric(stats[i])
    except:
        continue
        
# dropping columns with no information
for col in stats:
    if stats[col].isnull().sum() == len(stats):
        stats = stats.drop(col, axis = 1)
        
# adding trailing P/E
stats["trailingPE"] = stats["forwardPE"]*stats["forwardEps"]/stats["trailingEps"]

# Storing in SQL

## Sectors

In [8]:
for i in full_stk.sector.unique():
    my_cursor.execute("INSERT INTO sectors (sector) VALUES (%s)",(i,))
    db.commit()

## Stocks

In [9]:
# fetching sectors and sector_ids
my_cursor.execute("SELECT sector_id, sector FROM sectors")
sectors = []
for i in my_cursor:
    sectors.append(i)
    
# creating dataframe of data
sectors = pd.DataFrame(sectors, columns = ["sector_id", "sector"])

# adding sector_ids to stock list
full_stk = full_stk.merge(sectors, on = "sector", how = "outer")

In [10]:
# inserting stocks into stock table
for index, row in full_stk.iterrows():
    my_cursor.execute("INSERT INTO stock (ticker, company, sector_id) VALUES (%s,%s,%s)",
                      (full_stk.loc[index,"ticker"],
                       full_stk.loc[index,"company"],
                       int(full_stk.loc[index,"sector_id"]),)
                     )
db.commit()

## Affiliations

In [11]:
# fetching stock_id and ticker 
my_cursor.execute("SELECT stock_id, ticker from stock")
short_stk = []
for i in my_cursor:
    short_stk.append(i)

# creating dataframe of data
short_stk = pd.DataFrame(short_stk, columns = ["stock_id", "ticker"])

# adding stock_ids to stock list
full_stk = full_stk.merge(short_stk, on = "ticker", how = "outer")

In [12]:
# S&P500
for index, row in full_stk[full_stk.loc[:,"SP"] == 1].iterrows():
    my_cursor.execute("INSERT INTO sandp (stock_id, from_date) VALUES (%s,%s)",
                      (int(full_stk.loc[index, "stock_id"]),
                       today,
                      )
                     )
    
db.commit()

In [13]:
# Revolut
for index, row in full_stk[full_stk.loc[:,"Revolut"] == 1].iterrows():
    my_cursor.execute("INSERT INTO revolut (stock_id, from_date) VALUES (%s,%s)",
                      (int(full_stk.loc[index, "stock_id"]),
                       today,
                      )
                     )
    
db.commit()

## Metrics

In [14]:
# merging statistical information with stock information
stats = stats.merge(short_stk, on = "ticker", how = "left")

# filtering column
stats = stats.loc[:,["stock_id", "enterpriseToRevenue",
                     "enterpriseToEbitda", "enterpriseValue",
                     "profitMargins", "netIncomeToCommon",
                     "bookValue", "sharesOutstanding",
                     "sharesPercentSharesOut", "heldPercentInstitutions",
                     "heldPercentInsiders", "sharesShort",
                     "shortRatio", "floatShares", "forwardEps",
                     "trailingEps", "forwardPE", "trailingPE",
                     "pegRatio","priceToBook", "pricetoSales", "beta"]]

# dropping all null values
stats = stats.dropna()

In [15]:
for index, value in stats.iterrows():
    my_cursor.execute("INSERT INTO metrics (stock_id, date, enterpriseToRevenue,\
                                            enterpriseToEbitda, enterpriseValue,\
                                            profitMargins, netIncomeToCommon,\
                                            bookValue, sharesOutstanding,\
                                            sharesPercentSharesOut, heldPercentInstitutions,\
                                            heldPercentInsiders, sharesShort,\
                                            shortRatio, floatShares, forwardEps,\
                                            trailingEps, forwardPE, trailingPE,\
                                            pegRatio,priceToBook, pricetoSales, beta)\
                       VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                      (int(stats.loc[index, "stock_id"]),
                       today,
                       float(stats.loc[index, 'enterpriseToRevenue']),
                       float(stats.loc[index, 'enterpriseToEbitda']),
                       int(stats.loc[index, 'enterpriseValue']),
                       float(stats.loc[index, 'profitMargins']),
                       int(stats.loc[index, 'netIncomeToCommon']),
                       float(stats.loc[index, 'bookValue']),
                       int(stats.loc[index, 'sharesOutstanding']),
                       float(stats.loc[index, 'sharesPercentSharesOut']),
                       float(stats.loc[index, 'heldPercentInstitutions']),
                       float(stats.loc[index, 'heldPercentInsiders']),
                       int(stats.loc[index, 'sharesShort']),
                       float(stats.loc[index, 'shortRatio']),
                       int(stats.loc[index, 'floatShares']),
                       float(stats.loc[index, 'forwardEps']),
                       float(stats.loc[index, 'trailingEps']),
                       float(stats.loc[index, 'forwardPE']),
                       float(stats.loc[index, 'trailingPE']),
                       float(stats.loc[index, 'pegRatio']),
                       float(stats.loc[index, 'priceToBook']),
                       float(stats.loc[index, 'pricetoSales']),
                       float(stats.loc[index, 'beta']),
                      )
                     )
    
db.commit()
